In [22]:
import os
import boto3
import streamlit as st
from langchain_ollama import ChatOllama
import shutil
## We will be suing Titan Embeddings Model To generate Embeddings
from langchain_aws import BedrockEmbeddings
from langchain_aws import ChatBedrock

## Data Ingestion and Transformatioin
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader,PyPDFLoader


# Vector Embedding And Vector Store
from langchain_community.vectorstores import FAISS

## LLm Models
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA,ConversationalRetrievalChain

In [13]:
bedrock=boto3.client(service_name="bedrock-runtime")
bedrock_embeddings=BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock)

In [32]:
deepSeek=ChatBedrock(model_id="arn:aws:bedrock:ap-south-1:217522444118:inference-profile/apac.amazon.nova-pro-v1:0",client=bedrock,model_kwargs={'maxTokens':512,'model_provider':"amazon"})

In [26]:
deepSeek

ChatBedrock(client=<botocore.client.BedrockRuntime object at 0x168fa5700>, bedrock_client=<botocore.client.Bedrock object at 0x169484fb0>, model_id='arn:aws:bedrock:ap-south-1:217522444118:inference-profile/apac.amazon.nova-pro-v1:0', model_kwargs={}, max_tokens=512, beta_use_converse_api=True)

In [9]:
llama=ChatBedrock(model_id="meta.llama3-70b-instruct-v1:0",client=bedrock,
                model_kwargs={'max_gen_len':512})

In [33]:
from langchain_core.messages import HumanMessage

response = deepSeek.invoke([HumanMessage(content="Hello, explain quantum physics simply.")])
print(response.content)

ValueError: Model provider should be supplied when passing a model ARN as model_id

In [48]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage
import boto3
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

bedrock = boto3.client("bedrock-runtime", region_name="ap-south-1")

nova_llm = ChatBedrock(
    client=bedrock,
    model_id="openai.gpt-oss-120b-1:0", 
    model_kwargs={"max_completion_tokens": 512}
)

response = nova_llm.invoke([HumanMessage(content="Hello, explain quantum physics simply.")])
response = parser.invoke(response)
if "<reasoning>" in response:
    import re
    response = re.sub(r"<reasoning>.*?</reasoning>", "", response, flags=re.DOTALL).strip()

print(response)


Sure! Let’s think of the world as a giant stage where everything—particles, light, atoms—plays its part. Quantum physics is the script that tells us how the tiniest actors behave. Here’s a simple, step‑by‑step picture:

---

## 1. **Tiny is a Whole new world**
- **Classical physics** (the physics of everyday life) works perfectly for things you can see: cars, planets, baseballs.
- **Quantum physics** governs things that are *tiny*—electrons, photons (particles of light), atoms, and even smaller bits of matter.

## 2. **Things are also waves**
- In the quantum world, particles act like **waves** too. This is called *wave‑particle duality*.
- Imagine throwing a pebble into a pond. The ripple spreads out like a wave. In quantum physics, an electron can spread out in a similar “wave” way, describing where it *might* be.

## 3. **Probability, not certainty**
- Because of the wave nature, we can’t say exactly where a particle is. We can only talk about the **probability** of finding it in a 